In [47]:
import numpy as np
import random

# правила игры
class SimpleEnv:
    
    # Определяем размер пространства состояний и действий  
    def __init__(self):
        self.state_space_size = 4  # Например, 4 различных состояния
        self.action_space_size = 2  # Например, 2 действия (0 и 1)
    
    # логикa изменения состояния и назначения награды
    def step(self, state, action):
        next_state = (state + 1) % self.state_space_size # следующий шаг - предыдущий до начального
        reward = 1 if action == 0 else -1 # Награда зависит от выбранного действия
        done = next_state == 0 # если вернулись в начальное положение
        return next_state, reward, done
    
    # начальное состояние среды
    def reset(self):
        return 0
    
    # Гибридная политика: используем Q-таблицу, если есть положительные значения, иначе случайный выбор
    def hybrid_policy(q_table, state):
        if np.max(q_table[state]) > 0:
            return np.argmax(q_table[state])# *random.choice([0, 1])
        else:
            return random.choice([0, 1])

def train(env, episodes=100, learning_rate=0.8, gamma=0.95):
    # Инициализация Q-таблицы нулями
    q_table = np.zeros((env.state_space_size, env.action_space_size))
    for episode in range(episodes):
        state = env.reset()
        done = False

        while not done:
            action = hybrid_policy(q_table, state)  # Выбор действия согласно гибридной политике
            next_state, reward, done = env.step(state, action)

            # Обновление Q-таблицы
            q_table[state, action] = (1 - learning_rate) * q_table[state, action] + \
                                     learning_rate * (reward + gamma * np.max(q_table[next_state]))
            state = next_state

    return q_table

# Создаем экземпляр среды и обучаем агента
env = SimpleEnv()
env.reset
q_table = train(env)

# Выводим обученную Q-таблицу
print("Q-таблица:")
print(q_table)

Q-таблица:
[[19.85555793  0.        ]
 [19.85738353  0.        ]
 [19.85918607 -0.96      ]
 [19.86096582  0.7954432 ]]


In [46]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import random

class QLearningWrapper(BaseEstimator):
    def __init__(self, env, episodes=100, learning_rate=0.8, gamma=0.95):
        self.env = env
        self.episodes = episodes
        self.learning_rate = learning_rate
        self.gamma = gamma

    def fit(self, X=None, y=None):
        q_table = np.zeros((self.env.state_space_size, self.env.action_space_size))

        for episode in range(self.episodes):
            state = self.env.reset()
            done = False

            while not done:
                action = hybrid_policy(q_table, state)  # Выбор действия согласно гибридной политике
                next_state, reward, done = self.env.step(state, action)

                # Обновление Q-таблицы
                q_table[state, action] = (1 - self.learning_rate) * q_table[state, action] + \
                                         self.learning_rate * (reward + self.gamma * np.max(q_table[next_state]))
                state = next_state

        self.q_table_ = q_table
        return self

    def score(self, X=None, y=None):
        return 0

# Создаем экземпляр среды
env = SimpleEnv()
env.reset

# Определяем диапазоны для подбора гиперпараметров
param_dist = {
    'episodes': [1, 10000],
    'learning_rate': np.arange(0.1, 1.0, 0.1),
    'gamma': np.arange(0.1, 1.0, 0.1)
}

# Создаем пустые массивы для данных X и y
X_dummy = np.empty((10, env.state_space_size))
y_dummy = np.empty(10)

random_search = RandomizedSearchCV(estimator=QLearningWrapper(env), param_distributions=param_dist, cv=3, n_iter=10)

random_search.fit(X=X_dummy, y=y_dummy)

best_params = random_search.best_params_
print("Лучшие параметры:", best_params)

# Получаем обученную модель с лучшими параметрами
best_q_table = random_search.best_estimator_

# Выводим обученную Q-таблицу
print("Обученная Q-таблица с лучшими параметрами:")
print(best_q_table.q_table_)

Лучшие параметры: {'learning_rate': 0.7000000000000001, 'gamma': 0.4, 'episodes': 10000}
Обученная Q-таблица с лучшими параметрами:
[[ 1.66666667 -0.7       ]
 [ 1.66666667  0.        ]
 [ 1.66666667  0.        ]
 [ 1.66666667 -0.65912   ]]
